# Smith Watermelon Algorithm

In [5]:
# Scoring Matrix

import itertools
import numpy as np

def matrix(a, b, match_score=1, gap_cost=2):
    H = np.zeros((len(a) + 1, len(b) + 1), np.int)

    for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
        match = H[i - 1, j - 1] + (match_score if a[i - 1] == b[j - 1] else - match_score)
        delete = H[i - 1, j] - gap_cost
        insert = H[i, j - 1] - gap_cost
        H[i, j] = max(match, delete, insert, 0)
    return H 

In [ ]:
import itertools
import numpy as np

def matrix(a, b, match_score=1, gap_cost=3):
    H = np.zeros((len(a) + 1, len(b) + 1), np.int)

    for i, j in itertools.product(range(1, H.shape[0]), range(1, H.shape[1])):
        match = H[i - 1, j - 1] + (match_score if a[i - 1] == b[j - 1] else - match_score)
        delete = H[i - 1, j] - gap_cost
        insert = H[i, j - 1] - gap_cost
        H[i, j] = max(match, delete, insert, 0)
    return H

In [2]:
# Backtracing

def traceback(H, b, b_='', old_i=0):
    # flip H to get index of **last** occurrence of H.max() with np.argmax()
    H_flip = np.flip(np.flip(H, 0), 1)
    i_, j_ = np.unravel_index(H_flip.argmax(), H_flip.shape)
    i, j = np.subtract(H.shape, (i_ + 1, j_ + 1))  # (i, j) are **last** indexes of H.max()
    if H[i, j] == 0:
        return b_, j
    b_ = b[j - 1] + '-' + b_ if old_i - i > 1 else b[j - 1] + b_
    return traceback(H[0:i, 0:j], b, b_, i)

In [ ]:
def traceback(H, b, b_='', old_i=0):
    H_flip = np.flip(np.flip(H,0),1)
    i_, j_ = np.unravel_index(H_flip.argmax(), H_flip.shape)
    i, j = np.subtract(H.shape, (i_ + 1, j_ + 1)) 
    if H[i, j] == 0:
        return b_, j
    b_ = b[j - 1] + '-' + b_ if old_i - i > 1 else b[j - 1] + b_
    return traceback(H[0:i, 0:j], b, b_, i)

In [3]:
# Calculating start- and end-index

def smith_waterman(a, b, match_score=3, gap_cost=2):
    a, b = a.upper(), b.upper()
    H = matrix(a, b, match_score, gap_cost)
    b_, pos = traceback(H, b)
    return pos, pos + len(b_)

In [6]:
# Testing

print(matrix('AATGCA', 'ATAGCCA'))

a, b = 'aatgca', 'atagcca'
H = matrix(a, b)
print(traceback(H, b))

a, b = 'AATGCA', 'ATAGCCA'
start, end = smith_waterman(a, b)
print(a[start:end])    

[[0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 1]
 [0 1 0 1 0 0 0 1]
 [0 0 2 0 0 0 0 0]
 [0 0 0 1 1 0 0 0]
 [0 0 0 0 0 2 1 0]
 [0 1 0 1 0 0 1 2]]
('at-ca', 0)
AATGC


C:\Users\vmani\AppData\Local\Temp\ipykernel_7828\3449954520.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  H = np.zeros((len(a) + 1, len(b) + 1), np.int)


# Needleman-Wunsch Algorithm

In [9]:
def print_matrix(mat):
    # Loop over all rows
    for i in range(0, len(mat)):
        print("[", end = "")
        # Loop over each column in row i
        for j in range(0, len(mat[i])):
            # Print out the value in row i, column j
            print(mat[i][j], end = "")
            # Only add a tab if we're not in the last column
            if j != len(mat[i]) - 1:
                print("\t", end = "")
        print("]\n")



In [12]:
# Use these values to calculate scores
gap_penalty = -1
match_award = 1
mismatch_penalty = -1

# Make a score matrix with these two sequences
seq1 = "ATTACA"
seq2 = "ATGCT"

# A function for making a matrix of zeroes
def zeros(rows, cols):
    # Define an empty list
    retval = []
    # Set up the rows of the matrix
    for x in range(rows):
        # For each row, add an empty list
        retval.append([])
        # Set up the columns in each row
        for y in range(cols):
            # Add a zero to each column in each row
            retval[-1].append(0)
    # Return the matrix of zeros
    return retval

# A function for determining the score between any two bases in alignment
def match_score(alpha, beta):
    if alpha == beta:
        return match_award
    elif alpha == '-' or beta == '-':
        return gap_penalty
    else:
        return mismatch_penalty

# The function that actually fills out a matrix of scores
def needleman_wunsch(seq1, seq2):
    
    # length of two sequences
    n = len(seq1)
    m = len(seq2)  
    
    # Generate matrix of zeros to store scores
    score = zeros(m+1, n+1)
   
    # Calculate score table
    
    # Fill out first column
    for i in range(0, m + 1):
        score[i][0] = gap_penalty * i
    
    # Fill out first row
    for j in range(0, n + 1):
        score[0][j] = gap_penalty * j
    
    # Fill out all other values in the score matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            # Calculate the score by checking the top, left, and diagonal cells
            match = score[i - 1][j - 1] + match_score(seq1[j-1], seq2[i-1])
            delete = score[i - 1][j] + gap_penalty
            insert = score[i][j - 1] + gap_penalty
            # Record the maximum score from the three possible scores calculated above
            score[i][j] = max(match, delete, insert)
            
            
    # Create variables to store alignment
    align1 = ""
    align2 = ""
    
    # Start from the bottom right cell in matrix
    i = m
    j = n
    
    # We'll use i and j to keep track of where we are in the matrix, just like above
    while i > 0 and j > 0: # end touching the top or the left edge
        score_current = score[i][j]
        score_diagonal = score[i-1][j-1]
        score_up = score[i][j-1]
        score_left = score[i-1][j]
        
        # Check to figure out which cell the current score was calculated from,
        # then update i and j to correspond to that cell.
        if score_current == score_diagonal + match_score(seq1[j-1], seq2[i-1]):
            align1 += seq1[j-1]
            align2 += seq2[i-1]
            i -= 1
            j -= 1
        elif score_current == score_up + gap_penalty:
            align1 += seq1[j-1]
            align2 += '-'
            j -= 1
        elif score_current == score_left + gap_penalty:
            align1 += '-'
            align2 += seq2[i-1]
            i -= 1

    # Finish tracing up to the top left cell
    while j > 0:
        align1 += seq1[j-1]
        align2 += '-'
        j -= 1
    while i > 0:
        align1 += '-'
        align2 += seq2[i-1]
        i -= 1
    
    # Since we traversed the score matrix from the bottom right, our two sequences will be reversed.
    # These two lines reverse the order of the characters in each sequence.
    align1 = align1[::-1]
    align2 = align2[::-1]
    
    print(align1)
    print(align2)
    return score

print_matrix(needleman_wunsch(seq1, seq2))

ATTACA
A-TGCT
[0	-1	-2	-3	-4	-5	-6]

[-1	1	0	-1	-2	-3	-4]

[-2	0	2	1	0	-1	-2]

[-3	-1	1	1	0	-1	-2]

[-4	-2	0	0	0	1	0]

[-5	-3	-1	1	0	0	0]



# Branch and Bound Algorithm

In [1]:
#!/bin/python3

import numpy as np

def get_minimum_penalty(x:str, y:str, pxy:int, pgap:int):

	# initializing variables
	i = 0
	j = 0
	
	# pattern lengths
	m = len(x)
	n = len(y)
	
	# table for storing optimal substructure answers
	dp = np.zeros([m+1,n+1], dtype=int) #int dp[m+1][n+1] = {0};

	# initialising the table
	dp[0:(m+1),0] = [ i * pgap for i in range(m+1)]
	dp[0,0:(n+1)] = [ i * pgap for i in range(n+1)]

	# calculating the minimum penalty
	i = 1
	while i <= m:
		j = 1
		while j <= n:
			if x[i - 1] == y[j - 1]:
				dp[i][j] = dp[i - 1][j - 1]
			else:
				dp[i][j] = min(dp[i - 1][j - 1] + pxy,
								dp[i - 1][j] + pgap,
								dp[i][j - 1] + pgap)
			j += 1
		i += 1
	
	# Reconstructing the solution
	l = n + m # maximum possible length
	i = m
	j = n
	
	xpos = l
	ypos = l

	# Final answers for the respective strings
	xans = np.zeros(l+1, dtype=int)
	yans = np.zeros(l+1, dtype=int)
	

	while not (i == 0 or j == 0):
		#print(f"i: {i}, j: {j}")
		if x[i - 1] == y[j - 1]:	
			xans[xpos] = ord(x[i - 1])
			yans[ypos] = ord(y[j - 1])
			xpos -= 1
			ypos -= 1
			i -= 1
			j -= 1
		elif (dp[i - 1][j - 1] + pxy) == dp[i][j]:
		
			xans[xpos] = ord(x[i - 1])
			yans[ypos] = ord(y[j - 1])
			xpos -= 1
			ypos -= 1
			i -= 1
			j -= 1
		
		elif (dp[i - 1][j] + pgap) == dp[i][j]:
			xans[xpos] = ord(x[i - 1])
			yans[ypos] = ord('_')
			xpos -= 1
			ypos -= 1
			i -= 1
		
		elif (dp[i][j - 1] + pgap) == dp[i][j]:	
			xans[xpos] = ord('_')
			yans[ypos] = ord(y[j - 1])
			xpos -= 1
			ypos -= 1
			j -= 1
		

	while xpos > 0:
		if i > 0:
			i -= 1
			xans[xpos] = ord(x[i])
			xpos -= 1
		else:
			xans[xpos] = ord('_')
			xpos -= 1
	
	while ypos > 0:
		if j > 0:
			j -= 1
			yans[ypos] = ord(y[j])
			ypos -= 1
		else:
			yans[ypos] = ord('_')
			ypos -= 1

	# Since we have assumed the answer to be n+m long,
	# we need to remove the extra gaps in the starting
	# id represents the index from which the arrays
	# xans, yans are useful
	id = 1
	i = l
	while i >= 1:
		if (chr(yans[i]) == '_') and chr(xans[i]) == '_':
			id = i + 1
			break
		
		i -= 1

	# Printing the final answer
	print(f"Minimum Penalty in aligning the genes = {dp[m][n]}")
	print("The aligned genes are:")
	# X
	i = id
	x_seq = ""
	while i <= l:
		x_seq += chr(xans[i])
		i += 1
	print(f"X seq: {x_seq}")

	# Y
	i = id
	y_seq = ""
	while i <= l:
		y_seq += chr(yans[i])
		i += 1
	print(f"Y seq: {y_seq}")

def test_get_minimum_penalty():
	
	# input strings
	gene1 = "AGGGCT"
	gene2 = "AGGCA"
	
	# initialising penalties of different types
	mismatch_penalty = 1
	gap_penalty = 2

	# calling the function to calculate the result
	get_minimum_penalty(gene1, gene2, mismatch_penalty, gap_penalty)

test_get_minimum_penalty()


Minimum Penalty in aligning the genes = 3
The aligned genes are:
X seq: AGGGCT
Y seq: A_GGCA
